# Ariel Data Challenge 2025 - 探索的データ解析（EDA）

## 概要
このノートブックでは、Ariel Data Challenge 2025のデータセットの初期探索を行います。

**チャレンジの目標**: Arielの光学機器から系外惑星の大気信号を抽出する

**データ形式**:
- **wl_1 から wl_283**: 283の波長帯域での分光値
- **sigma_1 から sigma_283**: 各波長に対応する不確実性
- **planet_id**: 各系外惑星観測の一意識別子

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# 日本語フォント設定（必要に応じて）
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.style.use('default')
sns.set_palette("husl")

# パス設定
DATA_PATH = Path('../')
NOTEBOOKS_PATH = Path('.')

print("ライブラリの読み込み完了")

## 1. データセット概要

In [ ]:
# サンプル提出ファイルを読み込んで期待される出力形式を理解
sample_submission = pd.read_csv(DATA_PATH / 'sample_submission.csv')

print(f"サンプル提出ファイルの形状: {sample_submission.shape}")
print(f"カラム数: {len(sample_submission.columns)}")
print(f"波長帯域数 (wl_*): {len([col for col in sample_submission.columns if col.startswith('wl_')])}")
print(f"不確実性帯域数 (sigma_*): {len([col for col in sample_submission.columns if col.startswith('sigma_')])}")

sample_submission.head()

In [ ]:
# 惑星IDの確認
print(f"ユニークな惑星ID数: {sample_submission['planet_id'].nunique()}")
print(f"惑星ID: {sample_submission['planet_id'].tolist()}")

## 2. 利用可能なデータファイル

コンペティションファイルを一覧表示して、利用可能な学習/テストデータを確認します。

In [ ]:
import subprocess
import os

# コンペティション内の利用可能ファイルを一覧表示
try:
    result = subprocess.run(['kaggle', 'competitions', 'files', 'ariel-data-challenge-2025'], 
                          capture_output=True, text=True, cwd=DATA_PATH)
    print("コンペティション内の利用可能ファイル:")
    print(result.stdout)
except Exception as e:
    print(f"ファイル一覧取得エラー: {e}")
    print("次のコマンドを実行してください: kaggle competitions files ariel-data-challenge-2025")

## 3. データ構造分析

ファイル一覧から、データセットには以下が含まれることがわかります:
- **校正データ**: ダーク、デッドピクセル、フラットフィールド、線形補正、読み出しノイズファイル
- **異なる機器**: AIRS-CH0およびFGS1
- **複数の校正セット**: 各機器でcalibration_0とcalibration_1
- **Parquet形式**: すべてのデータファイルが効率的なparquet形式

In [ ]:
# データ構造を理解するため小さなサンプルファイルをダウンロード
sample_file = 'test/1103775/FGS1_calibration_0/dark.parquet'

try:
    # 分析用に単一の小さなファイルをダウンロード
    result = subprocess.run(['kaggle', 'competitions', 'download', '-c', 'ariel-data-challenge-2025', 
                           '-f', sample_file], 
                          capture_output=True, text=True, cwd=DATA_PATH)
    print(f"ダウンロード結果: {result.stdout}")
    if result.stderr:
        print(f"エラー: {result.stderr}")
        
except Exception as e:
    print(f"サンプルファイルダウンロードエラー: {e}")

In [ ]:
# ローカルにあるファイルを確認
import os

def list_files_recursive(path):
    """ディレクトリ構造を再帰的に表示"""
    for root, dirs, files in os.walk(path):
        level = root.replace(str(path), '').count(os.sep)
        indent = ' ' * 2 * level
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            print(f'{subindent}{file}')

print("現在のディレクトリ構造:")
list_files_recursive(DATA_PATH)

## 4. チャレンジの理解

Arielミッションの文脈から:

1. **AIRS-CH0**: Ariel赤外分光器チャンネル0
2. **FGS1**: 精密誘導センサー1
3. **校正タイプ**:
   - **dark**: ダーク電流校正
   - **dead**: デッドピクセルマッピング
   - **flat**: フラットフィールド校正
   - **linear_corr**: 線形性補正
   - **read**: 読み出しノイズ特性化

このチャレンジでは、これらの校正ファイルを処理して正確な系外惑星大気スペクトルを抽出することが求められます。

In [ ]:
# 期待される出力形式の分析
wavelength_cols = [col for col in sample_submission.columns if col.startswith('wl_')]
sigma_cols = [col for col in sample_submission.columns if col.startswith('sigma_')]

print(f"波長カラム数: {len(wavelength_cols)}")
print(f"最初の波長カラム: {wavelength_cols[:5]}")
print(f"最後の波長カラム: {wavelength_cols[-5:]}")

print(f"\n不確実性(sigma)カラム数: {len(sigma_cols)}")
print(f"最初のsigmaカラム: {sigma_cols[:5]}")
print(f"最後のsigmaカラム: {sigma_cols[-5:]}")

In [ ]:
# サンプル値の分析
print("サンプル提出ファイルの統計:")
print("\n波長値 (wl_* カラム):")
wl_data = sample_submission[wavelength_cols]
print(f"  最小値: {wl_data.min().min():.3f}")
print(f"  最大値: {wl_data.max().max():.3f}")
print(f"  平均値: {wl_data.mean().mean():.3f}")
print(f"  標準偏差: {wl_data.std().mean():.6f}")

print("\n不確実性値 (sigma_* カラム):")
sigma_data = sample_submission[sigma_cols]
print(f"  最小値: {sigma_data.min().min():.3f}")
print(f"  最大値: {sigma_data.max().max():.3f}")
print(f"  平均値: {sigma_data.mean().mean():.3f}")
print(f"  標準偏差: {sigma_data.std().mean():.6f}")

## 5. 次のステップ

この初期探索に基づく今後の作業:

1. **学習データのダウンロード**: 実際の校正ファイルを取得してデータ構造を理解
2. **データ前処理**: 校正データを処理するパイプラインの開発
3. **信号抽出**: 惑星信号を抽出するアルゴリズムの実装
4. **モデル開発**: 分光値と不確実性を予測するモデルの構築
5. **検証**: 交差検証戦略の開発

このチャレンジでは、複雑な機器校正データから微弱な系外惑星大気信号を抽出するため、
高度な信号処理と機械学習技術が必要です。

In [ ]:
# サンプル提出形式の簡単な可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# 波長値のプロット
wl_values = sample_submission.iloc[0][wavelength_cols].values
ax1.plot(range(len(wl_values)), wl_values, 'b-', alpha=0.7)
ax1.set_title('サンプル波長値 (wl_*)')
ax1.set_xlabel('波長インデックス')
ax1.set_ylabel('値')
ax1.grid(True, alpha=0.3)

# 不確実性値のプロット
sigma_values = sample_submission.iloc[0][sigma_cols].values
ax2.plot(range(len(sigma_values)), sigma_values, 'r-', alpha=0.7)
ax2.set_title('サンプル不確実性値 (sigma_*)')
ax2.set_xlabel('波長インデックス')
ax2.set_ylabel('不確実性')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n注意: これらはサンプル提出ファイルのプレースホルダー値です。")
print("実際の分光データはもっと多様性と構造を示すでしょう。")